In [ ]:
pip install openpyxl

In [ ]:
pip install transformers sentencepiece


In [ ]:
pip install transformers==4.30.0

In [ ]:
pip install torch


In [1]:
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec

# Define the paths to your CSV files
hs_dictionary_path = r"C:\Users\juan-\Documents\Proyecto Hacku AI\Repositorio GitHub página\proyecto_integrador_equipo_27\Avance_2\HS code Catalogue_Processed.csv"
Validation_input_path = r"C:\Users\juan-\Documents\Proyecto Hacku AI\Repositorio GitHub página\proyecto_integrador_equipo_27\Avance_2\Validation Data.csv"

# Read the CSV files into DataFrames, specifying the encoding
try:
    hs_dictionary_df = pd.read_csv(hs_dictionary_path, encoding='utf-8')
except UnicodeDecodeError:
    hs_dictionary_df = pd.read_csv(hs_dictionary_path, encoding='latin1')

try:
    Validation_input_df = pd.read_csv(Validation_input_path, encoding='utf-8')
except UnicodeDecodeError:
    Validation_input_df = pd.read_csv(Validation_input_path, encoding='latin1')

# Print the first few rows to ensure the content is read correctly
print(hs_dictionary_df.head())
print(Validation_input_df.head(10))




      HS Code                                   Full description
0          01                           Chapter 1   Live animals
1       01.01             Live horses, asses, mules and hinnies.
2  0101.21.00  Live horses, asses, mules and hinnies. - > Hor...
3  0101.29.00  Live horses, asses, mules and hinnies. - > Hor...
4     0101.30   Live horses, asses, mules and hinnies. -> Asses:
                                      Raw_data_input  \
0  CERAMIC TABLEWARE HS CODE 691200 2688 BOXES PO...   
1  1X40HR SAID TO CONTAIN 13 PACKAGE SUDU5157624 ...   
2  KONTRAKT  QHBG038545 LEERCONTAINER EX CDN NURN...   
3               MUEBLES Y ESTANTERIAS HS CODE 940310   
4  CHEMICAL PRODUCTS NOT SUBJECT TO IMDG CODEIMBE...   
5  1X20DC. SAID TO CONTAIN 21 PKGS IRON STEEL IRO...   
6  BITUMINOUS WATERPROOFING  MEMBRANEPRODUCT NAME...   
7            AUTOMOTIVE SPARE PARTS HS CODE 87085099   
8  NET WEIGHT 21500 KG; GROSS VOLUME 68 M3 GIRASO...   
9  2 PACAKGESBOS AUTOMOTIVE360 KGMPRODUCTS1 056 M.

In [2]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, BertTokenizer, BertModel
import nltk
from nltk.corpus import words, stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
import torch
from collections import Counter

# Download necessary NLTK data
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load the translation model (use Spanish-to-English model)
model_name = 'Helsinki-NLP/opus-mt-es-en'  # Spanish to English model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Load BERT model for attention scoring
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Get the English words corpus and Spanish stopwords
english_words = set(words.words())
spanish_stopwords = set(stopwords.words('spanish'))
english_stopwords = set(stopwords.words('english'))

# Add your custom stopwords to the Spanish stopwords
additional_stopwords = {
    'abv', 'accessories', 'aduana', 'agent', 'ancho', 'approximately', 'approx', 'ata', 'atd', 'attn', 'attention',
    'awb', 'b', 'bag', 'bags', 'barge', 'batch', 'big', 'bill', 'bl', 'bol', 'bonded', 'bottle', 'bottles',
    'box', 'boxes', 'broker', 'bulk', 'bundle', 'bundles', 'cargo', 'carrier', 'carton', 'cartons', 'case', 'cases',
    'cb', 'cbm', 'celsius', 'center', 'certificate', 'certification', 'cfr', 'charge', 'charges', 'cif', 'cip', 'circa',
    'claim', 'class', 'classification', 'clearance', 'cl', 'closing', 'cm', 'cmr', 'cntr', 'code', 'codes', 'collect',
    'collection', 'commercial', 'commodity', 'consolidated', 'consolidator', 'consignee', 'consignor', 'contain',
    'container', 'containers', 'containing', 'contains', 'contact', 'coo', 'cross', 'ctr', 'ctpat', 'cube', 'cubic',
    'customs', 'cut-off', 'damage', 'dangerous', 'dap', 'date', 'dated', 'dc', 'ddp', 'deadline', 'declared',
    'declares', 'declaration', 'degr', 'degrees', 'delay', 'delivery', 'demurrage', 'description', 'destination',
    'detention', 'dg', 'diameter', 'dim', 'dimension', 'dimensions', 'discharge', 'distribution', 'dock',
    'documentation', 'dt', 'dtd', 'duty', 'each', 'eccn', 'email', 'entry', 'eta', 'etd', 'event', 'exception',
    'excise', 'export', 'exportation', 'exw', 'fas', 'fax', 'fcl', 'feeder', 'fees', 'feu', 'flat', 'flight', 'fob',
    'forwarder', 'free', 'freight', 'gp', 'gross', 'gst', 'handling', 'hazardous', 'hbl', 'hc', 'height', 'high',
    'hr', 'hs', 'hs-code', 'hscode', 'hts', 'id', 'imdg', 'imo', 'import', 'importation', 'impression', 'impresion',
    'incoterms', 'inch', 'inches', 'inspection', 'insurance', 'intermodal', 'inv', 'invoice', 'invoices', 'iso',
    'item', 'items', 'iva', 'jumbo', 'kg', 'kgs', 'lading', 'largo', 'lcl', 'length', 'license', 'load', 'loaded',
    'loading', 'logistics', 'loose', 'loss', 'lot', 'lote', 'm3', 'manifest', 'marks', 'mbl', 'measurement',
    'measurements', 'medium', 'meter', 'metric', 'milestone', 'mm', 'mode', 'mt', 'multimodal', 'ncm', 'net',
    'netweight', 'no', 'non-hazardous', 'non-stackable', 'note', 'notes', 'notify', 'number', 'numbers', 'nvocc',
    'of', 'only', 'open', 'order', 'origin', 'packaging', 'packing', 'pallet', 'pallets', 'palletized', 'party',
    'payable', 'pc', 'pcs', 'permit', 'phytosanitary', 'pickup', 'piece', 'pieces', 'pkg', 'pkgs', 'pl', 'plt',
    'plts', 'po', 'port', 'prepaid', 'qty', 'quantity', 'quota', 'rack', 'rail', 'reefer', 'reel', 'reels', 'ref',
    'reference', 'report', 'restriction', 'rfc', 'roll', 'rolls', 'routing', 'sack', 'sacks', 'said', 'sailing',
    'scac', 'schedule', 'seal', 'seals', 'serial', 'sheets', 'shipment', 'shipper', 'shipping', 'sin', 'size',
    'sizes', 'small', 'stackable', 'status', 'stc', 'storage', 'stowed', 'stuffing', 'surcharges', 'survey',
    'system', 'tank', 'tare', 'tariff', 'tax', 'temperature', 'temp', 'terminal', 'teu', 'time', 'to', 'ton',
    'tons', 'top', 'total', 'tracking', 'tracing', 'trade', 'transit', 'transload', 'transhipment', 'transport',
    'truck', 'un', 'unit', 'unstuffing', 'valuation', 'value', 'vat', 'vessel', 'volume', 'voyage', 'warehouse',
    'weight', 'width', 'x', 'zone'
}


stop_words = spanish_stopwords.union(additional_stopwords)
stop_words = stop_words.union(english_stopwords)

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

# Function to translate text to English
def translate_to_english(text, tokenizer, model):
    translated = model.generate(**tokenizer.prepare_seq2seq_batch([text], return_tensors="pt"))
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Function to lemmatize, remove nonsense words, and clean text
def lemmatize_and_clean(text):
    words = text.split()
    cleaned_words = []
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        if lemma in english_words and lemma not in stop_words:
            cleaned_words.append(lemma)
    return cleaned_words  # Return list of cleaned words

# Function to get top tokens based on frequency and attention scores
def get_top_tokens(text, top_n=3):
    inputs = bert_tokenizer(text, return_tensors='pt')
    outputs = bert_model(**inputs)
    
    # Get the attention scores (mean pooling over token embeddings)
    attention_scores = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    
    # Create a dictionary of tokens and their corresponding attention scores
    tokens_with_scores = {token_id: score for token_id, score in zip(inputs['input_ids'][0], attention_scores)}
    
    # Filter out special tokens and get full tokens without subword prefixes
    filtered_tokens_scores = {}
    
    for token_id, score in tokens_with_scores.items():
        if token_id not in [bert_tokenizer.pad_token_id, bert_tokenizer.cls_token_id, bert_tokenizer.sep_token_id]:
            token_str = bert_tokenizer.decode([token_id]).replace("##", "")
            filtered_tokens_scores[token_str] = score
    
    # Count occurrences of each token in the cleaned words list for additional scoring
    word_counts = Counter(lemmatize_and_clean(text))
    
    # Combine frequency counts with attention scores for final ranking
    combined_scores = {token: word_counts[token] + filtered_tokens_scores.get(token, 0) for token in filtered_tokens_scores}
    
    # Sort by combined score and get top N tokens
    top_tokens = sorted(combined_scores.items(), key=lambda item: item[1], reverse=True)[:top_n]
    
    return [token for token, _ in top_tokens]

# Full pipeline function
def process_text(text):
    try:
        # Step 1: Preprocess
        preprocessed_text = preprocess_text(text)
        
        # Step 2: Translate
        translated_text = translate_to_english(preprocessed_text, tokenizer, model)
        
        # Step 3: Lemmatize, remove nonsense words, and clean
        cleaned_words_list = lemmatize_and_clean(translated_text)
        
        # Convert cleaned words list back to string for attention scoring
        final_cleaned_text = " ".join(cleaned_words_list)

        # Step 4: Get top tokens based on frequency and attention scores
        top_tokens = get_top_tokens(final_cleaned_text)

        return final_cleaned_text, top_tokens

    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return "", []  # Return empty string and empty list in case of error

# Example usage:
if __name__ == "__main__":
    sample_text_1 = "Perro vivo que come dogchow raza bulldog"
    processed_text_1, important_tokens_1 = process_text(sample_text_1)
    
    print("Processed Text:", processed_text_1)
    print("Top Tokens:", important_tokens_1)

    sample_text_2 = "CERAMIC TABLEWARE 2688 BOXES PO 5 39548"
    processed_text_2, important_tokens_2 = process_text(sample_text_2)

    print("Processed Text:", processed_text_2)
    print("Top Tokens:", important_tokens_2)

C:\Users\juan-\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\juan-\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juan-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juan-\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\juan-\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\juan-\AppData\Local\Pac

Processed Text: live dog eating bulldog breed
Top Tokens: ['bulldog', 'live', 'breed']
Processed Text: ceramic tableware
Top Tokens: ['ceramic', 'ware', 'table']


In [3]:
# Apply the full pipeline to the 'Validation_input' column
validation_cleaned_input = Validation_input_df.copy()
validation_cleaned_input['Validation_input_cleaned'] = Validation_input_df['Validation_input'].dropna().apply(process_text)

# Export the cleaned DataFrame to an Excel file
output_file = 'cleaned_validation_input.xlsx'
validation_cleaned_input.to_excel(output_file, index=False)

In [ ]:
process_text('NEW PNEUMATIC RADIAL TIREs 10')

In [ ]:
# Get the English words corpus and Spanish stopwords
english_words = set(words.words())
spanish_stopwords = set(stopwords.words('spanish'))

# Convert both to sorted lists for easier manipulation
english_words_list = sorted(list(english_words))
spanish_stopwords_list = sorted(list(spanish_stopwords))

# Create a pandas DataFrame from both lists
df = pd.DataFrame({
    'English_Words': pd.Series(english_words_list),
    'Spanish_Stopwords': pd.Series(spanish_stopwords_list)
})

# Save the DataFrame to an Excel file
output_file = 'words_and_stopwords.xlsx'
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")

In [ ]:
# Check downloaded words and stopwords
print(f"Number of English words: {len(english_words)}")
print(f"Number of Spanish stopwords: {len(spanish_stopwords)}")